In [46]:
from IPython.display import clear_output
!pip install keras-ocr

import pytesseract
import keras_ocr
import easyocr
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import os

In [1]:
!pip freeze > requirements_ocr.txt


In [3]:
from IPython.display import FileLink
FileLink('requirements_ocr.txt')

/kaggle/working/requirements_ocr.txt

In [48]:
#wczytanie danych
filelist_all = []
#localhost path
localhost_path = ""
online_path = "/kaggle/input"
for dirname, _, filenames in os.walk(online_path):
    for filename in filenames:
        filelist_all.append(os.path.join(dirname, filename))

In [56]:
%%time
#przetwarzanie po 5 plików z powodu limitów pamięci 
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
pipline = keras_ocr.pipeline.Pipeline() #Creting a pipline
df_full = pd.DataFrame()
for filelist in chunker(filelist_all, 5):
    print(filelist)
    # keras init
    
    images = [ keras_ocr.tools.read(i) for i in filelist]
    #keras ocr
    kerasocr_preds = pipline.recognize(images)
    #easyocr init
    text_reader = easyocr.Reader(['en', 'pl'])
    def filename_from_path(filepath):
        filename = filepath.split("/")[-1]
        return filename
    
    keras_predicted_ocr = []
    for prediction in kerasocr_preds:
        kerasocr_text = ""
        for text, box in prediction:
            kerasocr_text += " "+text
        keras_predicted_ocr.append(kerasocr_text)
    
    pytess_predicted_ocr = []
    for filepath in filelist:
        pytessocr_text = pytesseract.image_to_string(filepath)
        pytessocr_text = pytessocr_text.replace("\n", " ")
        pytess_predicted_ocr.append(pytessocr_text)

    easyocr_predicted_ocr = []
    for filepath in filelist:
        results = text_reader.readtext(filepath)
        easyocr_text = ""
        for (bbox, text, prob) in results:
            easyocr_text += " "+text
        easyocr_predicted_ocr.append(easyocr_text)
    
    # for ax , image,  prediction in zip(axs , images , kerasocr_preds):
    ocr_text = ""
    df_part = pd.DataFrame()
    for keras_prediction, easyocr_prediction, pytess_prediction, filepath in zip(keras_predicted_ocr, easyocr_predicted_ocr, pytess_predicted_ocr, filelist):
        temp = pd.DataFrame(
            {
                'filename': [filename_from_path(filepath)],
                'filepath': [filepath],
                'serwis': [filepath.split('/')[-2]],
                'kerasocr': [keras_prediction],
    #             'kerasocr_word_count': ["1"], #todo
                'easyocr': [easyocr_prediction],
    #             'easyocr_word_count': ["1"], #todo
                'pytessocr': [easyocr_prediction],
    #             'pytessocr_word_count': ["1"], #todo
            }
        )
        df_part = pd.concat([df_part, temp])
    df_full = pd.concat([df_full, df_part])


Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
['/kaggle/input/ads-files-extended/onet/8deba946-26b2-4769-8b0e-84a2451b57e6.jpg', '/kaggle/input/ads-files-extended/onet/9d1bca23-7dd7-44e9-b000-42ac374d41b9.jpg', '/kaggle/input/ads-files-extended/onet/d13fe224-d966-4082-8602-5fbe1824d62d.jpg', '/kaggle/input/ads-files-extended/onet/4c063f6e-7e89-4800-a7c1-cd595180389d.jpg', '/kaggle/input/ads-files-extended/onet/3318700e-de18-4400-9ace-cbe0fb21cd20.jpg']
['/kaggle/input/ads-files-extended/onet/925b2587-0dac-4ada-acfd-289dd9322e3e.jpg', '/kaggle/input/ads-files-extended/onet/f813ee80-1ee5-49a1-abeb-664684ff5b9c.jpg', '/kaggle/input/ads-files-extended/onet/d71c8ea6-fbed-414d-8ae1-3f6c08b6a0aa.jpg', '/kaggle/input/ads-files-extended/onet/844b4142-1028-49ea-b251-dd126f8a31cb.jpg', '/kaggle/input/ads-files-extended/onet/ec6a1e3d-0360-4d74-8dee-f0a58b9f9153.jpg']
['/kaggle/input/ads-files-extended/onet/f7ef0c56-c53b-49a0-953f-15c93b9007b2.jpg', '/ka

In [57]:
df_full

,filename,filepath,serwis,kerasocr,easyocr,pytessocr
0,8deba946-26b2-4769-8b0e-84a2451b57e6.jpg,/kaggle/input/ads-files-extended/onet/8deba946...,onet,reklama aries krynicazdroj doskonata lokaliza...,REKLANA Aries Krynica-Zdrój Doskonała lokaliz...,REKLANA Aries Krynica-Zdrój Doskonała lokaliz...
0,9d1bca23-7dd7-44e9-b000-42ac374d41b9.jpg,/kaggle/input/ads-files-extended/onet/9d1bca23...,onet,reklama only 2 have yov moves l 1q155,REKLANA 2 2 16:155 oNLY hAVE MoVes Yov,REKLANA 2 2 16:155 oNLY hAVE MoVes Yov
0,d13fe224-d966-4082-8602-5fbe1824d62d.jpg,/kaggle/input/ads-files-extended/onet/d13fe224...,onet,reklama f s z k,REKLANA,REKLANA
0,4c063f6e-7e89-4800-a7c1-cd595180389d.jpg,/kaggle/input/ads-files-extended/onet/4c063f6e...,onet,reklama sposobow 5 dzieki ktorym dorosli sie ...,REKLANA 5 sposobów dzięki którym dorośli uczą...,REKLANA 5 sposobów dzięki którym dorośli uczą...
0,3318700e-de18-4400-9ace-cbe0fb21cd20.jpg,/kaggle/input/ads-files-extended/onet/3318700e...,onet,reklama iphone taniej kodem z rabatowyme auto...,VFKa iPhone TANIEJ Z KODEM RABATOWYM! Autoryz...,VFKa iPhone TANIEJ Z KODEM RABATOWYM! Autoryz...
...,...,...,...,...,...,...
0,ReklamaFacebookOrange.png,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,stylowa polska sponsorowane przyszosc projekt...,Stylowa Polska Sponsorowane Zainwestuj w przy...,Stylowa Polska Sponsorowane Zainwestuj w przy...
0,ReklamFacebookaOrlen2.PNG,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,ssa industry boc sponsorowane comiesieczne dy...,SSA Industry Sponsorowane Polacy otrzymują co...,SSA Industry Sponsorowane Polacy otrzymują co...
0,ReklamaFacebookTVP.png,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,wolvero sponsorowane teraz jest najlepszy pol...,WOLVERO Sponsoroware Teraz jest najlepszy cza...,WOLVERO Sponsoroware Teraz jest najlepszy cza...
0,ReklamaFacebookOrlen.PNG,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,queenie bell sponsorowane koncern daje kazdem...,Queenie Bell Sponsorowane Koncern energetyczn...,Queenie Bell Sponsorowane Koncern energetyczn...


In [58]:
df_full.reset_index(inplace=True)
df_full.drop('index', inplace=True, axis=1)

In [59]:
df_full

,filename,filepath,serwis,kerasocr,easyocr,pytessocr
0,8deba946-26b2-4769-8b0e-84a2451b57e6.jpg,/kaggle/input/ads-files-extended/onet/8deba946...,onet,reklama aries krynicazdroj doskonata lokaliza...,REKLANA Aries Krynica-Zdrój Doskonała lokaliz...,REKLANA Aries Krynica-Zdrój Doskonała lokaliz...
1,9d1bca23-7dd7-44e9-b000-42ac374d41b9.jpg,/kaggle/input/ads-files-extended/onet/9d1bca23...,onet,reklama only 2 have yov moves l 1q155,REKLANA 2 2 16:155 oNLY hAVE MoVes Yov,REKLANA 2 2 16:155 oNLY hAVE MoVes Yov
2,d13fe224-d966-4082-8602-5fbe1824d62d.jpg,/kaggle/input/ads-files-extended/onet/d13fe224...,onet,reklama f s z k,REKLANA,REKLANA
3,4c063f6e-7e89-4800-a7c1-cd595180389d.jpg,/kaggle/input/ads-files-extended/onet/4c063f6e...,onet,reklama sposobow 5 dzieki ktorym dorosli sie ...,REKLANA 5 sposobów dzięki którym dorośli uczą...,REKLANA 5 sposobów dzięki którym dorośli uczą...
4,3318700e-de18-4400-9ace-cbe0fb21cd20.jpg,/kaggle/input/ads-files-extended/onet/3318700e...,onet,reklama iphone taniej kodem z rabatowyme auto...,VFKa iPhone TANIEJ Z KODEM RABATOWYM! Autoryz...,VFKa iPhone TANIEJ Z KODEM RABATOWYM! Autoryz...
...,...,...,...,...,...,...
198,ReklamaFacebookOrange.png,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,stylowa polska sponsorowane przyszosc projekt...,Stylowa Polska Sponsorowane Zainwestuj w przy...,Stylowa Polska Sponsorowane Zainwestuj w przy...
199,ReklamFacebookaOrlen2.PNG,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,ssa industry boc sponsorowane comiesieczne dy...,SSA Industry Sponsorowane Polacy otrzymują co...,SSA Industry Sponsorowane Polacy otrzymują co...
200,ReklamaFacebookTVP.png,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,wolvero sponsorowane teraz jest najlepszy pol...,WOLVERO Sponsoroware Teraz jest najlepszy cza...,WOLVERO Sponsoroware Teraz jest najlepszy cza...
201,ReklamaFacebookOrlen.PNG,/kaggle/input/ads-files-extended/facebook/Rekl...,facebook,queenie bell sponsorowane koncern daje kazdem...,Queenie Bell Sponsorowane Koncern energetyczn...,Queenie Bell Sponsorowane Koncern energetyczn...


In [60]:
df_full.to_csv('ads_files_ocr.csv', index=False)

In [61]:

from IPython.display import FileLink
FileLink('ads_files_ocr.csv')

/kaggle/working/ads_files_ocr.csv

In [ ]:
# keras init
pipline = keras_ocr.pipeline.Pipeline() #Creting a pipline
images = [ keras_ocr.tools.read(i) for i in filelist]

In [ ]:
#keras ocr
kerasocr_preds = pipline.recognize(images)

In [ ]:
#easyocr init
text_reader = easyocr.Reader(['en', 'pl'])

In [ ]:
def filename_from_path(filepath):
    filename = filepath.split("/")[-1]
    return filename
    

In [ ]:
keras_predicted_ocr = []
for prediction in kerasocr_preds:
    kerasocr_text = ""
    for text, box in prediction:
        kerasocr_text += " "+text
    keras_predicted_ocr.append(kerasocr_text)

In [ ]:
pytess_predicted_ocr = []

for filepath in filelist:
    pytessocr_text = pytesseract.image_to_string(filepath)
    pytessocr_text = pytessocr_text.replace("\n", " ")
    pytess_predicted_ocr.append(pytessocr_text)


In [ ]:
easyocr_predicted_ocr = []
for filepath in filelist:
    results = text_reader.readtext(filepath)
    easyocr_text = ""
    for (bbox, text, prob) in results:
        easyocr_text += " "+text
    easyocr_predicted_ocr.append(easyocr_text)

In [ ]:
# for ax , image,  prediction in zip(axs , images , kerasocr_preds):
ocr_text = ""
df = pd.DataFrame()
for keras_prediction, easyocr_prediction, pytess_prediction, filepath in zip(keras_predicted_ocr, easyocr_predicted_ocr, pytess_predicted_ocr, filelist):
    temp = pd.DataFrame(
        {
            'filename': [filename_from_path(filepath)],
            'filepath': [filepath],
            'kerasocr': [keras_prediction],
#             'kerasocr_word_count': ["1"], #todo
            'easyocr': [easyocr_prediction],
#             'easyocr_word_count': ["1"], #todo
            'pytessocr': [easyocr_prediction],
#             'pytessocr_word_count': ["1"], #todo
        }
    )
    df = pd.concat([df, temp])



In [ ]:
df.reset_index(inplace=True)
df.drop('index', inplace=True, axis=1)

In [ ]:
df

In [ ]:
# df.to_csv('fake_ads_ocr.csv', index=False)

In [ ]:

# from IPython.display import FileLink
# FileLink('fake_ads_ocr.csv')